In [9]:
from py2sambvca.py2sambvca import py2sambvca as p2s

input_xyz = 'xyz_structures//IRMOF-1.xyz'

buried_vol = p2s(
    xyz_filepath=input_xyz,
    sphere_center_atom_ids=[],
    z_ax_atom_ids=[],
    xz_plane_atoms_ids=[]
    )